In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, LSTM, Dropout, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Twitter_sentiment.csv', encoding='ISO-8859-1', header=None, names=['target', 'id', 'date', 'flag', 'user', 'text'],)
df = df.sample(n=100000, random_state=42)
df['target'] = df['target'].replace({0: 0, 4: 1})

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Twitter_sentiment.csv'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import re
import string

def preprocess_text(text):
    """
    Preprocesses the input text by removing URLs, special characters, and converting to lowercase.
    """
    # Convert to string if bytes-like object
    if isinstance(text, bytes):
        text = text.decode('utf-8', 'ignore')

    # Remove URLs
    text = re.sub(r'http\S+', '', text)

    # Remove punctuation and special characters
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'[^\x00-\x7F]+', '', text)

    # Convert to lowercase
    text = text.lower()

    return text

In [ ]:
df['text'] = df['text'].apply(preprocess_text)
df.head()


In [ ]:
train_size = int(len(df) * 0.8)
train_texts = df['text'][:train_size]
train_labels = df['target'][:train_size]
test_texts = df['text'][train_size:]
test_labels = df['target'][train_size:]

In [ ]:
from textblob import TextBlob
from textblob.tokenizers import WordTokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
import nltk
nltk.download('punkt')
from textblob import TextBlob
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenize train texts
train_sequences = []
for text in train_texts:
    blob = TextBlob(text)
    tokens = blob.words
    train_sequences.append(tokens)

# Tokenize test texts
test_sequences = []
for text in test_texts:
    blob = TextBlob(text)
    tokens = blob.words
    test_sequences.append(tokens)

# Convert tokens to sequences
tokenizer = Tokenizer(num_words=5000, lower=True)
tokenizer.fit_on_texts(train_sequences)

train_data = tokenizer.texts_to_sequences(train_sequences)
test_data = tokenizer.texts_to_sequences(test_sequences)

# Pad sequences
max_len = 100
train_data = pad_sequences(train_data, maxlen=max_len)
test_data = pad_sequences(test_data, maxlen=max_len)


In [ ]:
input_layer = Input(shape=(100,))
embedding_layer = Embedding(5000, 128, input_length=100)(input_layer)
lstm_layer = LSTM(64, dropout=0.2, recurrent_dropout=0.2)(embedding_layer)
dropout_layer = Dropout(0.5)(lstm_layer)
lr_layer = Dense(1, activation='sigmoid')(dropout_layer)
lr_model = Model(inputs=input_layer, outputs=lr_layer)

lstm_layer_2 = LSTM(64, dropout=0.2, recurrent_dropout=0.2)(embedding_layer
dropout_layer_2 = Dropout(0.5)(lstm_layer_2)
concat_layer = concatenate([dropout_layer, dropout_layer_2])
lr_layer_2 = Dense(1, activation='sigmoid')(concat_layer)

lstm_layer_3 = LSTM(64, dropout=0.2, recurrent_dropout=0.2)(embedding_layer)
dropout_layer_3 = Dropout(0.5)(lstm_layer_3)
concat_layer = concatenate([dropout_layer, dropout_layer_3])
lr_layer_3 = Dense(1, activation='sigmoid')(concat_layer)
lr_lstm_model = Model(inputs=input_layer, outputs=lr_layer_3)

In [ ]:
lr_model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy'])
lr_model.fit(train_data, train_labels, validation_split=0.1, epochs=5, batch_size=32)

lr_lstm_model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy'])
lr_lstm_model.fit(train_data, train_labels, validation_split=0.1, epochs=5, batch_size=32)


In [ ]:
# Define the maximum length of the input sequence
max_len = 100
# Preprocess the input string
input_text = "he is worst player"
preprocessed_text = preprocess_text(input_text)

# Tokenize and pad the input sequence
input_sequence = tokenizer.texts_to_sequences([preprocessed_text])
input_sequence = pad_sequences(input_sequence, maxlen=max_len)

# Make predictions using the lr_model
lr_predictions = lr_model.predict(input_sequence)
lr_predictions = np.round(lr_predictions).flatten()

# Make predictions using the lr_lstm_model
lr_lstm_predictions = lr_lstm_model.predict(input_sequence)
lr_lstm_predictions = np.round(lr_lstm_predictions).flatten()

# Print the predicted sentiment labels
threshold = 0.5
if lr_lstm_predictions >= threshold:
    sentiment="positive"
else:
    sentiment="negative"
#print("lr_model predictions:", lr_predictions)
print("lr_lstm_model predictions:", lr_lstm_predictions)
print("the given input string is ",sentiment)

In [ ]:
# Assuming you have the trained models lr_model and lr_lstm_model

# Predict labels using lr_model
lr_predictions = lr_model.predict(test_data)
lr_predictions = (lr_predictions >= 0.5).astype(int)  # Apply threshold for binary classification

# Predict labels using lr_lstm_model
lr_lstm_predictions = lr_lstm_model.predict(test_data)
lr_lstm_predictions = (lr_lstm_predictions >= 0.5).astype(int)  # Apply threshold for binary classification


In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# Calculate precision
precision_lr = precision_score(test_labels, lr_predictions)
precision_lr_lstm = precision_score(test_labels, lr_lstm_predictions)

# Calculate recall
recall_lr = recall_score(test_labels, lr_predictions)
recall_lr_lstm = recall_score(test_labels, lr_lstm_predictions)

# Calculate F1 score
f1_lr = f1_score(test_labels, lr_predictions)
f1_lr_lstm = f1_score(test_labels, lr_lstm_predictions)

print("Precision (lr_model):", precision_lr)
print("Precision (lr_lstm_model):", precision_lr_lstm)
print("Recall (lr_model):", recall_lr)
print("Recall (lr_lstm_model):", recall_lr_lstm)
print("F1 Score (lr_model):", f1_lr)
print("F1 Score (lr_lstm_model):", f1_lr_lstm)

In [ ]:
import matplotlib.pyplot as plt

# Train lr_model and store accuracy history
lr_history = lr_model.fit(train_data, train_labels, validation_split=0.1, epochs=5, batch_size=32)

# Train lr_lstm_model and store accuracy history
lr_lstm_history = lr_lstm_model.fit(train_data, train_labels, validation_split=0.1, epochs=5, batch_size=32)

# Plot accuracy over epochs for lr_model
plt.plot(lr_history.history['accuracy'])
plt.plot(lr_history.history['val_accuracy'])
plt.title('lr_model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'])
plt.show()

# Plot accuracy over epochs for lr_lstm_model
plt.plot(lr_lstm_history.history['accuracy'])
plt.plot(lr_lstm_history.history['val_accuracy'])
plt.title('lr_lstm_model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'])
plt.show()


In [ ]:
from sklearn.metrics import classification_report

# Generate classification report for lr_model
classification_report_lr = classification_report(test_labels, lr_predictions)
print("Classification Report (lr_model):")
print(classification_report_lr)

# Generate classification report for lr_lstm_model
classification_report_lr_lstm = classification_report(test_labels, lr_lstm_predictions)
print("Classification Report (lr_lstm_model):")
print(classification_report_lr_lstm)


In [ ]:
# Saving the model
lr_model.save('lr_model.h5')
lr_lstm_model.save('lr_lstm_model.h5')